In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle

from preprocessing_util import *

### April 2, 2024
### Preprocessing algorithm:
1. Subset NOTEEVENTS.csv by choosing a "HADM_ID" for each "SUBJECT_ID" by selection on lowest value of "CHARTDATE" ("NOTEEVENTS.csv")
2. Output A. Subset NOTEEVENTS.csv "HADM_ID" (drop everyone without notes, drop CATEGORY='Discharge summary'). Keep "CATEGORY" as a column
3. Output B. Left = DIAGNOSES_ICD.csv, Right="D_ICD_DIAGNOSES.csv" (just "ICD9_CODE","SHORT_TITLE"). Key="ICD9_CODE". left join
4. Joining Output A & B.
    1. Set A. Set of HADM_IDs from OutputA
    2. Row by row in Output B. Check if "HADM_ID" is in Set A. If yes, add to Counter.
    3. Counter/Dictionary: {"ICD9_CODE": Count of unique HADM_IDs from SetA with that code} (use Counter in Python)
    4. Output: {"58281": 10, "2762": 100} -> {"TB pneumonia-oth test: 10, ...} Sorted by value in dict
    5. Eventually: Choose top 10 or so ICD9 and join w/ output A 

In [ ]:
notes = pd.read_csv('../../MIMIC-III/NOTEEVENTS.csv')

In [ ]:
notes.columns

In [ ]:
# sort all of the notes by chart date so that we are only keeping the row of data with the smallest chart date value
notes = notes.sort_values(by='CHARTDATE')
notes

Note: There are some rows of data where the HADM_ID is nan. I am going to ignore those by dropping rows of data with nan HADM_ID.

In [ ]:
# drop rows of data where the hospital admission ID is nan
notes = notes.dropna(subset=['HADM_ID'])
notes

In [ ]:
# find all the unique subjects
unique_subjects = set(notes['SUBJECT_ID'])
len(unique_subjects)

In [ ]:
# find the hospital admission id's that correspond to the first appearance for each subject
# the first appearance corresponds to the lowest value of CHARTDATE since we sorted it previously
first_hadm_ids = []

for subject in unique_subjects:
    # only considering a unique subject in the dataframe
    temp = notes[notes['SUBJECT_ID'] == subject]

    # identify the first (also the earliest according to CHARTDATE because we have sorted by that already)
    # HADM_ID for this subject
    first_hadm_ids.append(temp.iloc[0]['HADM_ID'])

first_hadm_ids

In [ ]:
len(first_hadm_ids)

In [ ]:
# subset notes to keep only the first hospital admission id for each patient based off of CHARTDATE
notes_subset = notes[notes['HADM_ID'].isin(first_hadm_ids)]

notes_subset

In [ ]:
# drop all notes that are discharge summaries
output_A = notes_subset[notes_subset['CATEGORY'] != 'Discharge summary']
output_A

In [55]:
# save output_A as a csv file
output_A.to_csv('output_A.csv', index=False)

In [ ]:
diagnoses_icd = pd.read_csv('../../MIMIC-III/DIAGNOSES_ICD.csv')
diagnoses_icd

In [ ]:
diagnoses_icd_key = pd.read_csv('../../MIMIC-III/D_ICD_DIAGNOSES.csv')
diagnoses_icd_key = diagnoses_icd_key.drop(columns=['LONG_TITLE', 'ROW_ID'])
diagnoses_icd_key

In [ ]:
# do a left join based on ICD9_CODE so that we append the short title to each row
output_B = diagnoses_icd.merge(diagnoses_icd_key, how='left', on='ICD9_CODE')
output_B

In [ ]:
# drop nan values for the short title column
output_B = output_B.dropna(subset=['SHORT_TITLE'])
output_B

In [54]:
# save output_B as a csv file
output_B.to_csv('output_B.csv', index=False)

In [49]:
# declare a counter so that we can keep track of which diagnoses appear the most often
# in the output_A dataframe
diagnosis_counter = Counter()

In [ ]:
# go through each row in output_B; if the HADM_ID appears in first_hadm_ids, update the value of the counter
for index, row in output_B.iterrows():
    if row['HADM_ID'] in first_hadm_ids:
        diagnosis_counter[row['SHORT_TITLE']] += 1

diagnosis_counter.most_common(10)

In [53]:
pickle.dump(diagnosis_counter, open('diagnosis_counter.p', 'wb'))

Take those top 10 categories and add them as columns to a dataframe.

In [ ]:
top_diagnoses = diagnosis_counter.most_common(10)

# create dataframe for recording whether each patient received a specific diagnosis
diagnoses_df = output_A[['HADM_ID', 'SUBJECT_ID']].drop_duplicates()

diagnoses_df

In [ ]:
# next 10 columns are whether they received a diagnosis for the top 10 most common diagnoses

diagnosis_names = [tup[0] for tup in diagnosis_counter.most_common(10)]

for diagnosis in diagnosis_names:
    l = []
    for hadm_id in diagnoses_df['HADM_ID']:
        subset = output_B[output_B['HADM_ID'] == hadm_id]

        subset = subset[subset['SHORT_TITLE'] == diagnosis]
        
        if len(subset) >= 1:
            l.append(1)
        else:
            l.append(0)
        
    diagnoses_df[diagnosis] = l

diagnoses_df

In [98]:
diagnoses_df.to_csv('diagnoses_df.csv', index=False)

### Baseline confounders gender and age

Join on the gender and age of all of patients then drop all of the patients under 18 years of age. Update output_A as the csv with this additional information.

In [ ]:
output_A = pd.read_csv('output_A.csv')
output_A

In [ ]:
# read the csv containing demographic information of the patient
patients = pd.read_csv('../../MIMIC-III/PATIENTS.csv')
patients

In [ ]:
# transform the F and M in GENDER to 0 and 1
patients['gender'] = patients['GENDER'].map({'F': 0, 'M': 1})
patients

In [ ]:
output_A = output_A.merge(patients[['SUBJECT_ID', 'gender']], how='left', on='SUBJECT_ID')
output_A

In [ ]:
output_A = output_A.merge(patients[['SUBJECT_ID', 'DOB']], how='left', on='SUBJECT_ID')
output_A

In [ ]:
output_A['age'] = calculate_age(output_A['CHARTDATE'], output_A['DOB'])
output_A

In [2]:
output_A = pd.read_csv('output_A.csv')

In [ ]:
# drop all patients that are younger than 18 and greater than 100
output_A = output_A[output_A['age'] > 18]
output_A = output_A[output_A['age'] < 100]
output_A

In [4]:
# save the dataframe back into a csv file
output_A.to_csv('output_A.csv', index=False)

### Notes Categories Investigation

Investigate which categories we can try to subset for the output_A dataframe and still have a sufficiently large sample size.

In [ ]:
output_A = pd.read_csv('output_A.csv')

In [ ]:
categories = set(output_A['CATEGORY'])
categories

In [ ]:
# pre-processing where we make a dictionary such that key is the hospital admission id and
# value is all of the cateogires that they have
hadm_categories = {}

unique_hadms = list(set(output_A['HADM_ID']))

for hadm in unique_hadms:
    subset = output_A[output_A['HADM_ID'] == hadm]
    hadm_categories[hadm] = list(set(subset['CATEGORY']))

hadm_categories

In [ ]:
# make a dictionary that saves the categories and resulting dataframe sizes
save = {}

categories = list(categories)

for i in range(len(categories)):
    for j in range(i+1, len(categories)):
        category1 = categories[i]
        category2 = categories[j]
        cnt = 0
        for hadm in hadm_categories:
            if category1 in hadm_categories[hadm] and category2 in hadm_categories[hadm]:
                cnt += 1

        save[(category1, category2)] = cnt

save = dict(sorted(save.items(), key=lambda item: item[1], reverse=True))

save